In [2]:
import cv2
from skimage import io, exposure
import numpy as np
import glob
import os

In [9]:
root = '/home/dhruv/Projects/cybergut/Data/Prof_Ambedkar_Sandhya_shared_data/cAcO2-h187-post-50-uMFORS_1/'
ptrn = r'*01 - 0 ms*.tif'

In [11]:
rois = glob.glob(os.path.join(root, 'ROI', '0 ms', '*zip'))

In [13]:
cd ../

/run/media/dhruv/Gryffindor/Project/Cybergut/Implementation


In [15]:
from utils import roi

In [16]:
import re

In [19]:
reg = re.compile(r'UV - FITC')

In [20]:
channels = ['TL-Brightfield - dsRed', 'UV - FITC', 'UV - DAPI']

In [73]:
for file in rois:
    bname = os.path.basename(file[:-4])
    fname = bname+'.tif'
    polies = roi.get_polygons(file)
    imgs = [cv2.imread(root+'/'+reg.sub(c, fname), cv2.IMREAD_ANYDEPTH) for c in channels]
    imgs = [cv2.normalize(img, None, 0, 2**16-1, norm_type=cv2.NORM_MINMAX) for img in imgs]
    for xy in polies:
        pts = np.vstack(xy).T.reshape(-1, 1, 2)
        for img in imgs:
            cv2.polylines(img, [pts], True, 2**16-1, 3)
    imgs = [cv2.resize(img, (512, 512), None, interpolation=cv2.INTER_AREA) for img in imgs]
    bimg = np.zeros((1024, 1024), np.uint16)
    bimg[0:512, 0:512] = imgs[0]
    bimg[0:512, 512:] = imgs[1]
    bimg[512:, 0:512] = imgs[2]
    cv2.imwrite(f'/tmp/{bname}.png', bimg)